<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="-unSkript-Runbooks-">unSkript Runbooks&nbsp;</h1>
<div class="alert alert-block alert-success"><strong> This runbook demonstrates how to find redundant trails in AWS using unSkript actions.</strong></div>
</center><center>
<h2 id="Finding-Redundant-Trails-in-AWS">Finding Redundant Trails in AWS</h2>
</center>
<h1 id="Steps-Overview">Steps Overview</h1>
<p>1. Finding Redundant Trails in AWS</p>

<h3 id="Finding-Redundant-Trails-in-AWS"><a id="1" target="_self" rel="nofollow"></a>Finding Redundant Trails in AWS</h3>
<p>Here we will use unSkript Finding Redundant Trails in AWS action. The AWS CloudTrail service allows developers to enable policies managing compliance, governance, and auditing of their AWS accounts. In addition, AWS CloudTrail offers logging, monitoring, and storage of any activity around actions related to your AWS structures. The service activates from the moment you set up your AWS account, and while it provides real-time activity visibility, it also means higher AWS costs. This action is used to find Redundant Trails in AWS.</p>
<blockquote>
<p><strong>Output parameters:</strong>&nbsp;<code>redundant_trails</code></p>
</blockquote>

In [40]:
##
##  Copyright (c) 2023 unSkript, Inc
##  All rights reserved.
##
from pydantic import BaseModel, Field
from typing import Tuple
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
import pprint


def aws_finding_redundant_trails_printer(output):
    if output is None:
        return
    pprint.pprint(output)


def aws_finding_redundant_trails(handle) -> Tuple:
    """aws_finding_redundant_trails Returns an array of redundant trails in AWS
        :type handle: object
        :param handle: Object returned by the task.validate(...) method.
        :rtype: Tuple with check status and list of redundant trails
    """
    result = []
    all_regions = aws_list_all_regions(handle)
    for reg in all_regions:
        try:
            ec2Client = handle.client('cloudtrail', region_name=reg)
            response = ec2Client.describe_trails()
            for glob_service in response["trailList"]:
                if glob_service["IncludeGlobalServiceEvents"] is True:
                    for i in result:
                        if i["trail_name"] == glob_service["Name"]:
                            i["regions"].append(reg)
                    else:
                        if not any(i["trail_name"] == glob_service["Name"] for i in result):
                            trail_dict = {}
                            trail_dict["trail_name"] = glob_service["Name"]
                            trail_dict["regions"] = [reg]
                            result.append(trail_dict)
        except Exception as e:
            pass

    if len(result) != 0:
        return (False, result)
    else:
        return (True, None)


task = Task(Workflow())
task.configure(printOutput=True)
task.configure(outputName="redundant_trails")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_finding_redundant_trails, lego_printer=aws_finding_redundant_trails_printer, hdl=hdl, args=args)

<h3 id="Modify-Output">Modify Output</h3>
<p>In this action, we sort the commands on the basis of parameters given by the user as follows,</p>
<ol>
<li><code>stop_multiregion_trail = true </code>To turn off multi-region tracking of cloud trail.
<ol>
<li><code>aws cloudtrail update-trail&nbsp;<span class="hljs-attr">--region</span> us-west-<span class="hljs-number">2</span> <span class="hljs-attr">--name</span> cc-test-trail --no-<span class="hljs-keyword">is</span>-multi-region-trail</code></li>
</ol>
</li>
<li><code>global_event_tractiong = true </code>To turn off the global service events tracking the issue of the cloud trail.
<ol>
<li><code>aws cloudtrail update-trail&nbsp;<span class="hljs-attr">--region</span> us-west-<span class="hljs-number">2</span> <span class="hljs-attr">--name</span> cc-test-trail --no-<span class="hljs-keyword">include</span>-<span class="hljs-keyword">global</span>-service-events</code></li>
</ol>
</li>
<li>For <code>stop_multiregion_trail = true and global_event_tractiong = true</code>&nbsp;we use both commands to update the redundant trails.</li>
</ol>
<p><strong>Output parameters:</strong>&nbsp;<code>command_list</code></p>

In [41]:
command_list = []
if stop_multiregion_trail and not global_event_tractiong:
    if not redundant_trails[0]:
        for i in redundant_trails[1]:
            command = "aws cloudtrail update-trail --region " + region + " --name " + i["trail_name"] + " --no-is-multi-region-trail"
            command_list.append(command)
elif not stop_multiregion_trail and global_event_tractiong:
    if not redundant_trails[0]:
        for i in redundant_trails[1]:
            for region_1 in i["regions"]:
                command = "aws cloudtrail update-trail --region " + region_1 + " --name " + i["trail_name"] + " --no-include-global-service-events"
                command_list.append(command)
elif stop_multiregion_trail and global_event_tractiong:
    if not redundant_trails[0]:
        for i in redundant_trails[1]:
            command_1 = "aws cloudtrail update-trail --region " + region + " --name " + i["trail_name"] + " --no-include-global-service-events"
            command_2 = "aws cloudtrail update-trail --region " + region + " --name " + i["trail_name"] + " --no-is-multi-region-trail"
            command_list.append(command_1)
            command_list.append(command_2)


<h3 id="Run-Command-via-AWS-CLI">Run Command via AWS CLI</h3>
<p>In this action, we execute the commands from the above actions to update the redundant cloud trails.</p>
<p><strong>&nbsp; &nbsp; &nbsp; &nbsp;Input parameters:</strong>&nbsp;<code>aws_command</code></p>
<p><strong>&nbsp; &nbsp; &nbsp; &nbsp;Output parameters:</strong>&nbsp;<code>updated_output</code></p>
<p>&nbsp;</p>

In [42]:
#
# Copyright (c) 2021 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
import pprint


from beartype import beartype
@beartype
def aws_execute_cli_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def aws_execute_cli_command(handle, aws_command: str) -> str:

    result = handle.aws_cli_command(aws_command)
    if result is None or result.returncode != 0:
        print(
            f"Error while executing command ({aws_command}): {result}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "aws_command": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "command_list",
    "iter_parameter": "aws_command"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(command_list)>0",
    "condition_result": true
    }''')
task.configure(outputName="updated_output")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_execute_cli_command, lego_printer=aws_execute_cli_command_printer, hdl=hdl, args=args)

### Conclusion
<p>This Runbook demonstrates the use of unSkript's AWS actions to find redundant trails in AWS and update the cloud trails. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io/" rel="noopener">https://us.app.unskript.io</a></p>